# 1. Install and Import Dependencies

In [1]:
#!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
#!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification #string to sequence of nums + load into NLP and transformer
import torch
import requests
from bs4 import BeautifulSoup #extract data
import re #regex to cleanup text

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

# 2. Instantiate Model

In [4]:
#instantiate the BERT model - get the tokenizer and model from BERT huggingface website
link = 'nlptown/bert-base-multilingual-uncased-sentiment'

tokenizer = AutoTokenizer.from_pretrained(link)
model = AutoModelForSequenceClassification.from_pretrained(link)


# 3. Encode and Calculate Sentiment

### Testing on a  small sample

In [24]:
my_review = 'It was good spot but couldve been better experience if the service was faster'
tokens = tokenizer.encode(my_review, return_tensors='pt')

In [25]:
result = model(tokens)

In [27]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.7396,  0.2251,  2.1218,  1.2335, -1.5565]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [21]:
result.logits #results are raw from the nn - sequence classifier represent the prob of the sentiments

tensor([[-1.7396,  0.2251,  2.1218,  1.2335, -1.5565]],
       grad_fn=<AddmmBackward0>)

In [22]:
# convert results into readable format
# plus 1 here helps because counting start from 0 (for 1 to 5 scale)
int(torch.argmax(result.logits))+1 

3

# 4. Collect Reviews

In [28]:
url = 'https://www.yelp.com/biz/social-brew-cafe-pyrmont'
url = 'https://www.yelp.com/biz/the-feed-co-table-and-tavern-chattanooga-4?osq=Restaurants'
r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [29]:
reviews

["We went to Feed for brunch yesterday and were blown away. Joel was our server and we will totally ask for him every time we go back. He recommended the hot pimento and honey chicken biscuit and it was phenomenal!!! My husband got his other recommendation, the wagyu smash burger. Ridiculously good!!! Food and bloody Mary's came out quickly. We will definitely be back!!!",
 'Great food and service! Our server Noal, he  was amazing!  He was funny and entertaining!',
 "Honestly this was one of the best meals I've had in Chattanooga. I met a coworker for lunch based on her recommendation and recommendation from several others. And boy they were not wrong!  The food was phenomenal. After going back and forth about several options, I finally settled on catfish with a corn salsa, mash potatoes and changed up the green beans for greens based on a recommendation from the waitress. (I wish I did the brussel sprouts because my coworker ordered them and they were beyond phenomenal!The catfish was

# 5. Load Reviews into DataFrame and Score

In [30]:
import numpy as np
import pandas as pd

In [31]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [32]:
df['review'].iloc[0]

"We went to Feed for brunch yesterday and were blown away. Joel was our server and we will totally ask for him every time we go back. He recommended the hot pimento and honey chicken biscuit and it was phenomenal!!! My husband got his other recommendation, the wagyu smash burger. Ridiculously good!!! Food and bloody Mary's came out quickly. We will definitely be back!!!"

In [33]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [34]:
sentiment_score(df['review'].iloc[1])

5

In [35]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [17]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,The food was delicious. The ricotta pancakes w...,4
3,Great place with delicious food and friendly s...,5
4,"Great service, lovely location, and really ama...",5
5,Great food amazing coffee and tea. Short walk ...,5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,It was ok. Had coffee with my friends. I'm new...,3
8,We came for brunch twice in our week-long visi...,4
9,I came to Social brew cafe for brunch while ex...,5


In [18]:
df['review'].iloc[3]

'Great place with delicious food and friendly staff. It is small but has outdoor seating and a relaxed ambiance. Perfect place to enjoy a cup of coffee. I am visiting Sydney for the first time but this place seems like is a local favorite.'